# Supervised Fine Tuning (SFT)

## Motivation

Supervised Fine-Tuning (SFT) is one of the most straightforward ways to improve the performance of large language models. Unlike preference-based methods like DPO, SFT works when we already have high-quality, labeled data—where each input is paired with a desired output. This makes it ideal for tasks where ground truth responses are available, such as summarization, question answering, or instruction following.

## How it works (High Level)

In SFT, we train the model to mimic the desired behavior by directly optimizing for correct outputs using standard supervised learning. The model sees input-output pairs and learns to produce the expected result. This method is simple and effective, especially when the training data is well-aligned with the task you want the model to perform. It’s often used as a first step before applying more advanced techniques like reinforcement learning or preference optimization.

## How it really works (Teacher Forcing)

SFT also uses **Teacher Forcing** during training. This means we feed the correct output (the target sequence) to the model, one token at a time, and calculate how likely the model is to generate each token. The training objective is to increase the probability of generating the correct tokens, step by step.

By learning directly from labeled examples, the model becomes better at producing responses that match the expected outputs. There's no comparison between alternatives like in DPO—just direct learning from ground truth sequences.


## Dataset

To use SFT, you need a dataset of **(input, output)** pairs. Each example should include a prompt or instruction and the correct response. These outputs are usually written by humans or curated carefully to reflect the desired behavior. The quality of this data is very important—better data leads to better model performance. Unlike DPO, there's no need for preference comparisons or rejected alternatives.

**Important Note: Use the notebook [Dataset-Creation-SFT.ipynb](Dataset-Creation-SFT.ipynb) to prepare DPO dataset before running this notebook.**

## Quantization

Training or running large language models often requires significant memory, which can be a challenge on limited hardware. **Quantization** helps by reducing the size of the model weights, typically from 16 or 32-bit floats down to 8-bit or even 4-bit integers. This makes the model smaller and faster, with minimal impact on performance in most cases. In this project, we use the **BitsAndBytes** library to apply quantization efficiently.

## Parameter Efficient Fine Tuning (PEFT) with Low Rank Adaptation (LoRA)

Fine-tuning large models from scratch can be expensive and slow. **PEFT** techniques aim to reduce this cost by updating only a small number of parameters. One popular method is **Low Rank Adaptation (LoRA)**, which injects small trainable matrices into the model's layers without changing the original weights. This allows for efficient fine-tuning with fewer resources. We use the **peft** library to implement LoRA in our experiments.

## Signal to Noise Ratio (SNR) with Spectrum

Not all layers in a model contribute equally to learning during fine-tuning. **Signal to Noise Ratio (SNR)** helps identify which layers are more useful to focus on by comparing meaningful signal to background noise in the weight updates. This can guide efficient adaptation and avoid overfitting. We use the **spectrum** library to compute and analyze SNR during training.

## Metrics

To evaluate our fine-tuned language model, we use several metrics:

- **accuracy**: Measures how often the model's output matches the expected output exactly.
- **bleu**: A precision-based metric that compares n-gram overlap between generated and reference texts, commonly used in translation tasks.
- **rouge1, rouge2, rougeL**: Recall-based metrics that check how many unigrams (rouge1), bigrams (rouge2), or longest common sequences (rougeL) overlap with the target.
- **bertscore_precision, bertscore_recall, bertscore_f1**: Use BERT embeddings to compare the similarity of generated and reference sentences on a deeper, semantic level.
- **avg_levenshtein**: Measures the average number of edits (insertions, deletions, substitutions) needed to change the model output into the reference text, useful for judging closeness in form.

Together, these metrics give a balanced view of both surface-level accuracy and deeper semantic alignment.

## Note on Runtime

Due to limited compute and time, the experiments and metrics in this guide were run in under 2 hours. In practice, fine-tuning large models usually requires multiple days or even weeks to achieve optimal results.

# Imports

In [ ]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
import yaml
from peft import LoraConfig, get_peft_model, PeftModel
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset, DatasetDict, Dataset
import Levenshtein
import evaluate
import numpy as np
import json
from transformers import set_seed
import random
import pandas as pd

# Print Versions

In [1]:
import sys
import platform
import os
import subprocess


def print_env_info():
    print("\n🌱 Welcome to your Environment Info Report! 🌱\n")

    # Python & OS
    print(f"🐍 Python:       {platform.python_implementation()} {platform.python_version()}")
    print(f"💻 Platform:     {platform.system()} {platform.release()} ({platform.machine()})\n")

    # CUDA & GPU via PyTorch
    try:
        import torch
        cuda_avail = torch.cuda.is_available()
        print(f"🚀 CUDA Available: {cuda_avail}")
        if cuda_avail:
            print(f"   • CUDA Version:   {torch.version.cuda}")
            print(f"   • cuDNN Version:  {torch.backends.cudnn.version()}")
            n_gpus = torch.cuda.device_count()
            print(f"   • GPU Count:      {n_gpus}")
            for i in range(n_gpus):
                print(f"     - GPU {i}:      {torch.cuda.get_device_name(i)}")
    except ImportError:
        print("🚫 PyTorch not installed, skipping CUDA/GPU info")

    # nvcc (if available)
    try:
        out = subprocess.check_output(['nvcc', '--version'], stderr=subprocess.STDOUT)
        release = [l for l in out.decode().splitlines() if "release" in l]
        print(f"\n📦 nvcc: {release[-1].strip()}")
    except Exception:
        print("\n📦 nvcc: not found in PATH")

    # nvidia-smi
    try:
        out = subprocess.check_output([
            'nvidia-smi',
            '--query-gpu=name,driver_version,memory.total',
            '--format=csv,noheader'], stderr=subprocess.DEVNULL
        ).decode().strip().splitlines()
        print("\n📊 nvidia-smi info:")
        for line in out:
            print("   " + line)
    except Exception:
        print("\n📊 nvidia-smi: not available")

    # Helper to show versions
    def show_ver(label, import_name=None):
        try:
            m = __import__(import_name or label)
            v = getattr(m, '__version__', None) or getattr(m, 'VERSION', None) or str(m)
            print(f"🔖 {label:<15} version: {v}")
        except ImportError:
            print(f"🔖 {label:<15} not installed")

    # Popular ML/LLM libraries
    print("\n📚 Library versions:")
    libs = [
        ('torch',      None),
        ('torchvision', None),
        ('torchaudio',  None),
        ('transformers', None),
        ('accelerate',  None),
        ('trl',         'trl'),
        ('peft',        'peft'),
        ('deepspeed',   None),
        ('bitsandbytes', None),
        ('datasets',    'datasets'),
        ('evaluate',    'evaluate'),
        ('tokenizers',  None),
        ('sentencepiece', None),
        ('huggingface_hub', None),
        ('numpy',       'numpy'),
        ('scipy',       'scipy'),
        ('pandas',      'pandas'),
        ('scikit-learn','sklearn'),
        ('wandb',       'wandb'),
        ('tensorboard', 'tensorboard'),
        ('mlflow',      'mlflow'),
    ]
    for label, name in libs:
        show_ver(label, name)

    # Conda env and key env vars
    print("\n🔧 Conda env:", os.getenv('CONDA_DEFAULT_ENV', '(none)'))
    important_vars = ['CUDA_HOME', 'CUDA_PATH', 'LD_LIBRARY_PATH', 'HF_HOME', 'HF_DATASETS_CACHE']
    print("\n🌐 Environment variables:")
    for var in important_vars:
        print(f"   - {var:<15} = {os.getenv(var, '')}")

    print("\n✨ All set! Keep growing and training with confidence! ✨\n")

print_env_info()


🌱 Welcome to your Environment Info Report! 🌱

🐍 Python:       CPython 3.12.7
💻 Platform:     Linux 6.11.0-26-generic (x86_64)

🚀 CUDA Available: True
   • CUDA Version:   12.4
   • cuDNN Version:  90100
   • GPU Count:      1
     - GPU 0:      NVIDIA GeForce RTX 4070 Laptop GPU

📦 nvcc: Cuda compilation tools, release 12.1, V12.1.105

📊 nvidia-smi info:
   NVIDIA GeForce RTX 4070 Laptop GPU, 570.133.07, 8188 MiB

📚 Library versions:
🔖 torch           version: 2.6.0+cu124
🔖 torchvision     version: 0.21.0+cu124
🔖 torchaudio      version: 2.6.0+cu124
🔖 transformers    version: 4.51.3
🔖 accelerate      version: 1.6.0
🔖 trl             version: 0.17.0


/home/electron/PycharmProjects/fine_tune_llm/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔖 peft            version: 0.15.2
🔖 deepspeed       not installed
🔖 bitsandbytes    version: 0.45.5
🔖 datasets        version: 3.6.0
🔖 evaluate        version: 0.4.3
🔖 tokenizers      version: 0.21.1
🔖 sentencepiece   version: 0.2.0
🔖 huggingface_hub version: 0.31.2
🔖 numpy           version: 2.2.5
🔖 scipy           version: 1.15.3
🔖 pandas          version: 2.2.3
🔖 scikit-learn    version: 1.6.1
🔖 wandb           version: 0.19.11
🔖 tensorboard     not installed
🔖 mlflow          not installed

🔧 Conda env: (none)

🌐 Environment variables:
   - CUDA_HOME       = 
   - CUDA_PATH       = 
   - LD_LIBRARY_PATH = 
   - HF_HOME         = 
   - HF_DATASETS_CACHE = 

✨ All set! Keep growing and training with confidence! ✨



# Seed and Env

In [2]:
WANDB_TOKEN = "WANDB-TOKEN-GOES-HERE"
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
SEED = 69

In [ ]:
os.environ["WANDB_API_KEY"] = WANDB_TOKEN
os.environ["WANDB_PROJECT"] = "lora_fine_tune_math_question_answer_spectrum"

In [3]:
def seed_everything(seed: int = SEED):
    # Python built-in
    random.seed(seed)

    # Numpy
    np.random.seed(seed)

    # PyTorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True  # Slower but reproducible
    torch.backends.cudnn.benchmark = False

    # Environment variables
    os.environ["PYTHONHASHSEED"] = str(seed)

    # Hugging Face Transformers
    set_seed(seed)

In [4]:
seed_everything(SEED)

# Quantization

In [5]:
def setup_quantization_4bit(
    quant_type: str = "nf4",
    use_double_quant: bool = True,
    compute_dtype=torch.float16,
) -> BitsAndBytesConfig:
    """
    Create a BitsAndBytesConfig for 4-bit quantization.
    """
    return BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type=quant_type,
        bnb_4bit_use_double_quant=use_double_quant,
        bnb_4bit_compute_dtype=compute_dtype,
    )

# Load Model and Tokenizer

In [6]:
def load_model_and_tokenizer(
    model_name: str,
    bnb_config: BitsAndBytesConfig,
    device_map: str = "auto",
):
    """
    Load the pretrained model in 4-bit and its tokenizer.
    Adjust tokenizer/model configs for chat formatting and training.
    """
    # Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    # Ensure we have a pad token
    if tokenizer.pad_token_id is None:
        tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})
    # Model
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map=device_map,
    )
    # Required for 4-bit finetuning
    model.config.use_cache = False
    # Resize token embeddings if we added a pad token
    model.resize_token_embeddings(len(tokenizer))
    return model, tokenizer

# PEFT (LoRA) + Spectrum (SNR)

In [20]:
def apply_lora_peft(
    model: torch.nn.Module,
    r: int = 16,
    alpha: int = 32,
    dropout: float = 0.05,
    target_modules: list = ["q_proj", "v_proj"],
    spectrum_yaml_path: str = "/home/electron/PycharmProjects/fine_tune_llm/spectrum/"
                    "snr_results_TinyLlama-TinyLlama-v1.1_unfrozenparameters_10percent.yaml",
    spectrum: bool = False,
):
    """
    Wrap the base model with PEFT + LoRA adapters.
    """
    if spectrum:
        MODEL_NAME = "TinyLlama/TinyLlama_v1.1"
        
        # 1) Load the YAML
        if not os.path.isfile(spectrum_yaml_path):
            raise FileNotFoundError(
                f"Spectrum YAML not found at {spectrum_yaml_path}. "
                "Please run Spectrum to generate it first."
            )
        with open(spectrum_yaml_path, "r") as yf:
            data = yaml.safe_load(yf)
    
        # 2) Extract the regex list
        target_modules = data.get("unfrozen_parameters")
        if not target_modules or not isinstance(target_modules, list):
            raise ValueError(
                f"No 'unfrozen_parameters' list found in {spectrum_yaml_path}."
            )
    
    print('Target Lora Modules: ', target_modules)
    
    lora_cfg = LoraConfig(
        r=r,
        lora_alpha=alpha,
        target_modules=target_modules,
        lora_dropout=dropout,
        bias="none",
        task_type="CAUSAL_LM",
    )
    return get_peft_model(model, lora_cfg)

# SFT Config

In [8]:
def make_training_args(
    output_dir: str = "sft_tinnyllama",
    learning_rate: float = 1e-4,
    per_device_train_batch_size: int = 2,
    per_device_eval_batch_size: int = 2,
    num_train_epochs: int = 1,
    logging_steps: int = 20,
    eval_steps: int = 200,
    save_steps: int = 200,
    gradient_accumulation_steps: int = 2,
    bf16: bool = False,
    fp16: bool = True,
    report_to: str = "wandb",
    run_name: str = "tinyllama-sft",
    packing: bool = True,
):
    return SFTConfig(
        output_dir=output_dir,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_eval_batch_size,
        num_train_epochs=num_train_epochs,
        learning_rate=learning_rate,
        logging_steps=logging_steps,
        eval_strategy="steps",
        eval_steps=eval_steps,
        save_strategy="steps",
        save_steps=save_steps,
        gradient_accumulation_steps=gradient_accumulation_steps,
        bf16=bf16,
        fp16=fp16,
        optim="paged_adamw_32bit",
        lr_scheduler_type="cosine",
        warmup_ratio=0.03,
        load_best_model_at_end=True,
        greater_is_better=False,               # minimize eval loss
        report_to=report_to,
        run_name=run_name,
        packing=packing,
        save_total_limit=3,
    )

# SFT Trainer

In [9]:
def build_trainer(
    model: torch.nn.Module,
    tokenizer,
    train_dataset,
    eval_dataset,
    training_args: SFTConfig,
) -> SFTTrainer:
    """
    Instantiate the TRL SFTTrainer.
    """
    return SFTTrainer(
        model=model,
        processing_class=tokenizer,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
    )

# Inference

In [ ]:
def run_inference(
    model: torch.nn.Module,
    tokenizer,
    dataset: Dataset,
    max_length: int = 256,
    batch_size: int = 16,
    device: str = None,
) -> Dataset:
    device = device or (next(model.parameters()).device)
    model.eval()

    def gen_batch(batch):
        prompts = [
            "".join(m["content"] for m in msgs if m["role"] in ("system", "user"))
            for msgs in batch["messages"]
        ]
        enc = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            out = model.generate(
                **enc,
                max_new_tokens=max_length,
                eos_token_id=tokenizer.eos_token_id,
            )
        batch["prediction"] = tokenizer.batch_decode(out[:, enc["input_ids"].shape[-1]:], skip_special_tokens=True)
        return batch

    return dataset.map(gen_batch, batched=True, batch_size=batch_size)

# Evaluation

In [ ]:
def evaluate_generation(
    dataset_with_preds,
    reference_col: str = "messages",
    prediction_col: str = "prediction",
) -> dict:
    # Extract references
    refs = [
        "".join(m["content"] for m in sample[reference_col] if m["role"] == "assistant")
        for sample in dataset_with_preds
    ]
    preds = dataset_with_preds[prediction_col]

    # 1) Exact-match accuracy
    matches = [int(p.strip() == r.strip()) for p, r in zip(preds, refs)]
    accuracy = sum(matches) / len(matches)

    # 2) BLEU, ROUGE, BERTScore
    bleu_metric = evaluate.load("bleu")
    rouge_metric = evaluate.load("rouge")
    bert_metric = evaluate.load("bertscore")

    bleu = bleu_metric.compute(predictions=preds, references=[[r] for r in refs])["bleu"]
    rouge_scores = rouge_metric.compute(predictions=preds, references=refs)
    bert_scores = bert_metric.compute(predictions=preds, references=refs, lang="en")

    # 3) Average Levenshtein distance
    distances = [Levenshtein.distance(p, r) for p, r in zip(preds, refs)]
    avg_levenshtein = float(np.mean(distances))

    return {
        "accuracy": accuracy,
        "bleu": bleu,
        "rouge1": rouge_scores["rouge1"],
        "rouge2": rouge_scores["rouge2"],
        "rougeL": rouge_scores["rougeL"],
        "bertscore_precision": float(np.mean(bert_scores["precision"])),
        "bertscore_recall": float(np.mean(bert_scores["recall"])),
        "bertscore_f1": float(np.mean(bert_scores["f1"])),
        "avg_levenshtein": avg_levenshtein,
    }

# Load Dataset

In [11]:
def load_for_sft(data_dir: str = "./lora_processed_data") -> DatasetDict:
    files = {
        "train": os.path.join(data_dir, "train.jsonl"),
        "eval": os.path.join(data_dir, "eval.jsonl"),
        "test": os.path.join(data_dir, "test.jsonl")
    }
    ds = load_dataset("json", data_files=files)
    return ds

In [12]:
sft_ds = load_for_sft("lora_processed_data")
sft_ds

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 160028
    })
    eval: Dataset({
        features: ['messages'],
        num_rows: 20003
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 20004
    })
})

In [13]:
sft_ds['train'] = sft_ds['train'].shuffle(seed=SEED).select(range(10000))
sft_ds['eval'] = sft_ds['eval'].shuffle(seed=SEED).select(range(1000))
sft_ds['test'] = sft_ds['test'].shuffle(seed=SEED).select(range(1000))
sft_ds

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 10000
    })
    eval: Dataset({
        features: ['messages'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 1000
    })
})

In [14]:
sft_ds['test'][11]['messages']

[{'role': 'system',
  'content': 'Solve the given high school math problem by providing a clear explanation of each step leading to the final solution.\n\n    Provide a detailed breakdown of your calculations, beginning with an explanation of the problem and describing how you derive each formula, value, or conclusion. Use logical steps that build upon one another, to arrive at the final answer in a systematic manner.\n\n    # Steps\n\n    1. **Understand the Problem**: Restate the given math problem and clearly identify the main question and any important given values.\n    2. **Set Up**: Identify the key formulas or concepts that could help solve the problem (e.g., algebraic manipulation, geometry formulas, trigonometric identities).\n    3. **Solve Step-by-Step**: Iteratively progress through each step of the math problem, justifying why each consecutive operation brings you closer to the solution.\n    4. **Double Check**: If applicable, double check the work for accuracy and sense

In [15]:
sft_ds['test'][11]['messages']

[{'role': 'system',
  'content': 'Solve the given high school math problem by providing a clear explanation of each step leading to the final solution.\n\n    Provide a detailed breakdown of your calculations, beginning with an explanation of the problem and describing how you derive each formula, value, or conclusion. Use logical steps that build upon one another, to arrive at the final answer in a systematic manner.\n\n    # Steps\n\n    1. **Understand the Problem**: Restate the given math problem and clearly identify the main question and any important given values.\n    2. **Set Up**: Identify the key formulas or concepts that could help solve the problem (e.g., algebraic manipulation, geometry formulas, trigonometric identities).\n    3. **Solve Step-by-Step**: Iteratively progress through each step of the math problem, justifying why each consecutive operation brings you closer to the solution.\n    4. **Double Check**: If applicable, double check the work for accuracy and sense

# Fine Tuning

## Quantization Setup

In [17]:
# 1) Prepare 4-bit quantization config
bnb_config = setup_quantization_4bit()
bnb_config

BitsAndBytesConfig {
  "_load_in_4bit": true,
  "_load_in_8bit": false,
  "bnb_4bit_compute_dtype": "float16",
  "bnb_4bit_quant_storage": "uint8",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": true,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}

## Model and Tokenizer Setup

In [18]:
# 2) Load model & tokenizer
model, tokenizer = load_model_and_tokenizer(MODEL_NAME, bnb_config)
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear4bit(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), e

## LoRA Setup

In [22]:
# 3) Apply LoRA adapters
model = apply_lora_peft(model, spectrum=True)
model

Target Lora Modules:  ['^lm_head.weight$', '^model.embed_tokens.weight$', 'model.layers.21.mlp.down_proj', 'model.layers.1.mlp.down_proj', 'model.layers.2.mlp.gate_proj', 'model.layers.5.mlp.gate_proj', 'model.layers.7.mlp.up_proj', 'model.layers.3.mlp.up_proj', 'model.layers.12.self_attn.k_proj', 'model.layers.21.self_attn.k_proj', 'model.layers.13.self_attn.o_proj', 'model.layers.15.self_attn.o_proj', 'model.layers.0.self_attn.q_proj', 'model.layers.14.self_attn.q_proj', 'model.layers.1.self_attn.v_proj', 'model.layers.2.self_attn.v_proj']


/home/electron/PycharmProjects/fine_tune_llm/venv/lib/python3.12/site-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear4bit(i

## SFT Config Setup

In [23]:
# 4) Build training arguments
training_args = make_training_args()
training_args

SFTConfig(output_dir='sft_tinnyllama', overwrite_output_dir=False, do_train=False, do_eval=True, do_predict=False, eval_strategy=<IntervalStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=2, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=2, eval_accumulation_steps=None, eval_delay=0, torch_empty_cache_steps=None, learning_rate=0.0001, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1, max_steps=-1, lr_scheduler_type=<SchedulerType.COSINE: 'cosine'>, lr_scheduler_kwargs={}, warmup_ratio=0.03, warmup_steps=0, log_level='passive', log_level_replica='warning', log_on_each_node=True, logging_dir='sft_tinnyllama/runs/May15_22-11-34_electrion-legion', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=20, logging_nan_inf_filter=True, save_strategy=<SaveStrategy.STEPS: 'steps'>, save_steps=20

# Trainer Setup

In [24]:
# 5) Instantiate trainer
trainer = build_trainer(model, tokenizer, sft_ds['train'], sft_ds['eval'], training_args)
trainer

Converting train dataset to ChatML: 100%|█| 10000/10000 [00:00<00:00, 12167.15 e
Applying chat template to train dataset: 100%|█| 10000/10000 [00:00<00:00, 15337
Packing train dataset: 100%|███| 10000/10000 [00:00<00:00, 589650.79 examples/s]
Converting eval dataset to ChatML: 100%|█| 1000/1000 [00:00<00:00, 12351.59 exam
Applying chat template to eval dataset: 100%|█| 1000/1000 [00:00<00:00, 15220.74
Packing eval dataset: 100%|██████| 1000/1000 [00:00<00:00, 340917.17 examples/s]
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [21]:
# 6) Kick off training
trainer.train()

wandb: Currently logged in as: szamani to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
200,0.562600,0.548954
400,0.504800,0.521572
600,0.496400,0.508515
800,0.494100,0.500428
1000,0.463400,0.494781
1200,0.501100,0.491200
1400,0.499600,0.489170
1600,0.482600,0.488456


TrainOutput(global_step=1796, training_loss=0.531722470487412, metrics={'train_runtime': 2338.9691, 'train_samples_per_second': 3.071, 'train_steps_per_second': 0.768, 'total_flos': 4.575488604163277e+16, 'train_loss': 0.531722470487412})

In [22]:
# 7) Save adapter weights & final state
trainer.save_state()

In [23]:
trainer.model.save_pretrained(os.path.join(training_args.output_dir, "adapter_model"))

In [24]:
trainer.model.print_trainable_parameters()

trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044


# Full Dump of Model

In [ ]:
def export_merged_model_for_serving(
    base_model_name: str,
    adapter_dir: str,
    export_dir: str,
    tokenizer,
    bnb_config=None,  # optional quant config if your base is quantized
    device_map="auto",
):
    # 1) Load the base model in the same setup used for training
    if bnb_config:
        base = AutoModelForCausalLM.from_pretrained(
            base_model_name,
            quantization_config=bnb_config,
            device_map=device_map,
        )
    else:
        base = AutoModelForCausalLM.from_pretrained(
            base_model_name,
            device_map=device_map,
        )

    # 2) Wrap and load adapters
    peft_model = PeftModel.from_pretrained(
        base,
        adapter_dir,
        device_map=device_map,
    )

    # 3) Merge LoRA weights & unload adapter wrapper
    merged_model = peft_model.merge_and_unload()
    merged_model.config.use_cache = True  # set for inference

    # 4) Save the merged model + config.json + tokenizer
    merged_model.save_pretrained(export_dir)
    tokenizer.save_pretrained(export_dir)

    print(f"Full model saved to {export_dir}. You can now run:")
    print(f"  vllm serve {export_dir}")

In [ ]:
export_dir = os.path.join(training_args.output_dir, "merged_model_for_serving")
export_merged_model_for_serving(
    base_model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    adapter_dir=os.path.join(training_args.output_dir, "adapter_model"),
    export_dir=export_dir,
    tokenizer=tokenizer,
    bnb_config=bnb_config,            # if you used 4-bit quantization
    device_map="auto",
)

# Evaluation Setup

In [30]:
fresh_preds = run_inference(model, tokenizer, sft_ds['test'])
fresh_metrics = evaluate_generation(fresh_preds)
fresh_metrics

Parameter 'function'=<function run_inference.<locals>.gen_batch at 0x72f20c14fc40> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map: 100%|███████████████████████████| 1000/1000 [10:35<00:00,  1.57 examples/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'accuracy': 0.0,
 'bleu': 0.0007939185784332719,
 'rouge1': np.float64(0.11707655447961005),
 'rouge2': np.float64(0.05219114355057252),
 'rougeL': np.float64(0.0938309527964162),
 'bertscore_precision': 0.792242191016674,
 'bertscore_recall': 0.7308731575012207,
 'bertscore_f1': 0.7594701766371726,
 'avg_levenshtein': 797.449}

In [ ]:
fresh_preds['messages'][31]

In [ ]:
fresh_preds['prediction'][31]

# Reload Model

In [ ]:
def load_trained_or_base(
    model_name: str,
    adapter_path: str,
    load_finetuned: bool = True,
    device_map: str = "auto",
):
    # 1) Prepare quant config and load base model & tokenizer
    bnb_config = setup_quantization_4bit()
    model, tokenizer = load_model_and_tokenizer(model_name, bnb_config, device_map=device_map)

    if load_finetuned:
        # wrap base in PEFT and load saved adapter weights
        model = apply_lora_peft(model)
        model = PeftModel.from_pretrained(model, adapter_path, device_map=device_map)
    return model, tokenizer

In [ ]:
def load_base_model_non_quantized(
    model_name: str,
    device_map: str = "auto",
):
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    if tokenizer.pad_token_id is None:
        tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map=device_map)
    model.config.use_cache = True
    model.resize_token_embeddings(len(tokenizer))
    return model, tokenizer

# Evaluation After Reload

In [32]:
# 1) Off-the-shelf Non-quantized base
base_nq_model, base_nq_tokenizer = load_base_model_non_quantized(MODEL_NAME)
base_nq_preds = run_inference(base_nq_model, base_nq_tokenizer, sft_ds['test'])
base_nq_metrics = evaluate_generation(base_nq_preds)
base_nq_metrics

Map: 100%|█████████████████████████| 1000/1000 [2:00:44<00:00,  7.24s/ examples]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'accuracy': 0.0,
 'bleu': 0.03395986881810831,
 'rouge1': np.float64(0.22268675405019422),
 'rouge2': np.float64(0.09641640587021322),
 'rougeL': np.float64(0.16172563862058176),
 'bertscore_precision': 0.7793596202731132,
 'bertscore_recall': 0.7538000769615173,
 'bertscore_f1': 0.7653101402521133,
 'avg_levenshtein': 785.509}

In [37]:
base_nq_preds

Dataset({
    features: ['messages', 'prediction'],
    num_rows: 1000
})

In [47]:
base_nq_preds['messages'][31]

[{'role': 'system',
  'content': 'Solve the given high school math problem by providing a clear explanation of each step leading to the final solution.\n\n    Provide a detailed breakdown of your calculations, beginning with an explanation of the problem and describing how you derive each formula, value, or conclusion. Use logical steps that build upon one another, to arrive at the final answer in a systematic manner.\n\n    # Steps\n\n    1. **Understand the Problem**: Restate the given math problem and clearly identify the main question and any important given values.\n    2. **Set Up**: Identify the key formulas or concepts that could help solve the problem (e.g., algebraic manipulation, geometry formulas, trigonometric identities).\n    3. **Solve Step-by-Step**: Iteratively progress through each step of the math problem, justifying why each consecutive operation brings you closer to the solution.\n    4. **Double Check**: If applicable, double check the work for accuracy and sense

In [48]:
base_nq_preds['prediction'][31]

'numerical answer: 12\n\n    - Use clear and concise language, avoiding jargon or technical terms that may confuse the reader.\n    - Provide examples or visual aids to help illustrate the steps and formulas.\n    - Use a clear and readable font size and style, and avoid using too many lines or paragraphs.\n    - Use bullet points or numbered lists to organize the steps and formulas.\n    - Use a consistent formatting style, such as bold or italicized text, to make the solution easy to read.\n    - Provide a clear and concise explanation of each step, including any necessary assumptions or limitations.\n    - Use a consistent format for the solution, such as using a decimal point or rounding to the nearest integer.\n    - Use a consistent format for the answer, such as using a decimal point or rounding to the nearest integer.\n    - Use a consistent format for the solution, such as using a decimal point or rounding to the nearest integer.\n    - Use a consistent format for the answer, 

In [24]:
# 2) Off-the-shelf but quantized
base_model, base_tokenizer = load_trained_or_base(MODEL_NAME, adapter_path='', load_finetuned=False)
base_preds = run_inference(base_model, base_tokenizer, sft_ds['test'])
base_metrics = evaluate_generation(base_preds)
base_metrics

Map: 100%|███████████████████████████| 1000/1000 [10:53<00:00,  1.53 examples/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'accuracy': 0.0,
 'bleu': 0.032405014898393716,
 'rouge1': np.float64(0.19743619803845586),
 'rouge2': np.float64(0.08459908887520971),
 'rougeL': np.float64(0.1479884543575836),
 'bertscore_precision': 0.7803523952960968,
 'bertscore_recall': 0.7545850667953491,
 'bertscore_f1': 0.7661809774637223,
 'avg_levenshtein': 821.885}

In [25]:
base_preds['messages'][31]

[{'role': 'system',
  'content': 'Solve the given high school math problem by providing a clear explanation of each step leading to the final solution.\n\n    Provide a detailed breakdown of your calculations, beginning with an explanation of the problem and describing how you derive each formula, value, or conclusion. Use logical steps that build upon one another, to arrive at the final answer in a systematic manner.\n\n    # Steps\n\n    1. **Understand the Problem**: Restate the given math problem and clearly identify the main question and any important given values.\n    2. **Set Up**: Identify the key formulas or concepts that could help solve the problem (e.g., algebraic manipulation, geometry formulas, trigonometric identities).\n    3. **Solve Step-by-Step**: Iteratively progress through each step of the math problem, justifying why each consecutive operation brings you closer to the solution.\n    4. **Double Check**: If applicable, double check the work for accuracy and sense

In [26]:
base_preds['prediction'][31]

'ematics:\n    - Solve the problem by using the given math problem as a starting point.\n    - Use the given math problem as a guide to help you understand the problem better.\n    - Use the given math problem as a reference to check your work.\n    - Use the given math problem as a starting point to guide your own solution.\n    - Use the given math problem as a reference to check your own work.\n    - Use the given math problem as a starting point to guide your own solution.\n    - Use the given math problem as a reference to check your own work.\n    - Use the given math problem as a starting point to guide your own solution.\n    - Use the given math problem as a reference to check your own work.\n    - Use the given math problem as a starting point to guide your own solution.\n    - Use the given math problem as a reference to check your own work.\n    - Use the given math problem as a starting point to guide your own solution.\n    - Use the given math problem as a reference to c

In [27]:
ADAPTER_DIR = "./sft_tinnyllama/adapter_model"

# 3) Fine-tuned
ft_model, ft_tokenizer = load_trained_or_base(MODEL_NAME, ADAPTER_DIR, load_finetuned=True)
ft_preds = run_inference(ft_model, ft_tokenizer, sft_ds['test'])
ft_metrics = evaluate_generation(ft_preds)
ft_metrics

/home/electron/PycharmProjects/fine_tune_llm/venv/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/home/electron/PycharmProjects/fine_tune_llm/venv/lib/python3.12/site-packages/peft/peft_model.py:569: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.1.self_attn.q_proj.lora_B.defaul

{'accuracy': 0.0,
 'bleu': 0.032405014898393716,
 'rouge1': np.float64(0.19743619803845586),
 'rouge2': np.float64(0.08459908887520971),
 'rougeL': np.float64(0.1479884543575836),
 'bertscore_precision': 0.7803523952960968,
 'bertscore_recall': 0.7545850667953491,
 'bertscore_f1': 0.7661809774637223,
 'avg_levenshtein': 821.885}

In [28]:
ft_preds['messages'][31]

[{'role': 'system',
  'content': 'Solve the given high school math problem by providing a clear explanation of each step leading to the final solution.\n\n    Provide a detailed breakdown of your calculations, beginning with an explanation of the problem and describing how you derive each formula, value, or conclusion. Use logical steps that build upon one another, to arrive at the final answer in a systematic manner.\n\n    # Steps\n\n    1. **Understand the Problem**: Restate the given math problem and clearly identify the main question and any important given values.\n    2. **Set Up**: Identify the key formulas or concepts that could help solve the problem (e.g., algebraic manipulation, geometry formulas, trigonometric identities).\n    3. **Solve Step-by-Step**: Iteratively progress through each step of the math problem, justifying why each consecutive operation brings you closer to the solution.\n    4. **Double Check**: If applicable, double check the work for accuracy and sense

In [29]:
ft_preds['prediction'][31]

'ematics:\n    - Solve the problem by using the given math problem as a starting point.\n    - Use the given math problem as a guide to help you understand the problem better.\n    - Use the given math problem as a reference to check your work.\n    - Use the given math problem as a starting point to guide your own solution.\n    - Use the given math problem as a reference to check your own work.\n    - Use the given math problem as a starting point to guide your own solution.\n    - Use the given math problem as a reference to check your own work.\n    - Use the given math problem as a starting point to guide your own solution.\n    - Use the given math problem as a reference to check your own work.\n    - Use the given math problem as a starting point to guide your own solution.\n    - Use the given math problem as a reference to check your own work.\n    - Use the given math problem as a starting point to guide your own solution.\n    - Use the given math problem as a reference to c

In [ ]:
# 4) Print comparison
print("=== Non-Quantized Base Metrics ===")
for k, v in base_nq_metrics.items():
    print(f"{k}: {v:.4f}")

print("\n=== Quantized Base Metrics ===")
for k, v in base_q_metrics.items():
    print(f"{k}: {v:.4f}")

print("\n=== Fine-Tuned Quantized Metrics ===")
for k, v in ft_metrics.items():
    print(f"{k}: {v:.4f}")

In [ ]:
# Optional: Combined report
report = {
    k: {
        "base_non_quantized": base_nq_metrics[k],
        "base_quantized": base_q_metrics[k],
        "fine_tuned": ft_metrics[k],
    }
    for k in base_nq_metrics
}
df = pd.DataFrame(report).T
print("\n=== All Models Comparison ===")
print(df)